# Unit-testing with `unittest` & `pytest`

<hr>

## Basics of `unittest`

Well-tested code helps you:
- Find bugs earlier
- Iterate faster
- Debug more easily
- Design better code


Consider a script containing multiple functions to be deployed:

```python
# calc.py
def add(x,y):
    return x + y

def divide(x,y):
    if y == 0:
        raise ValueError('Can not divide by zero!')
    return x / y
```

****

One way to ensure that these functions work as intended is to import these functions into a script that runs unit tests on them:

```python
# test_calc.py
import unittest
import calc

class TestCalc(unittest.TestCase):
    # methods needs to named with 'test_<name>' for tests to run
    def test_add(self):
        # consider testing with random numbers and edge cases
        result = calc.add(10, 5)
        self.assertEqual(result, 15)
        
    def test_divide(self):
        # testing if the correct error was raised as intended
        self.assertRaises(ValueError, calc.divide, 10, 0)
        
        # same as above but a better way is to use a context manager when testing for errors
        with self.assertRaises(ValueError):
            calc.divide(10, 0)

# Run all unit tests
if __name__ == '__main__':
    unittest.main()
```

****

Using `setUp` and `tearDown` methods within the class will help to setup attributes and tear them down for each test.

Using `setUpClass` and `tearDownClass` runs once before and after all tests are complete.

```python
import unittest
from employee import Employee # import class Employee from employee.py

class TestEmployee(unittest.TestCase):
    @classmethod
    def setUpClass(cls):
        # run some costly setup code for e.g. populate a database
        
    @classmethod
    def tearDownClass(cls):
        # run some teardown code
    
    # needs to be uppercase 'U' and 'D' for setup/teardown
    def setUp(self):
        self.emp_1 = Employee('John', 'Doe' 1000)
        
    def tearDown(self):
        pass
    
    # write tests below
    def test_function(self):
        ...
```

****

Keeping tests isolated from external dependencies is important - code should be tested to run as intended and not to check if databases are working which is out of the code's scope.

One way to test such code without actually interacting with external dependencies is to do mocking.

```python
# Suppose a method called monthly_schedule exists in employee class
def test_monthly_schedule(self):
    # define a context manager and patch the function where it is actually used (i.e. employee)
    with patch('employee.requests.get') as mocked_get:
        # set return value expectations
        mocked_get.return_value.ok = True
        mocked_get.return_value.text = 'Success' # this will return 'Success' for the mock call
        
        # call function
        schedule = self.emp_1.monthly_schedule('May')
        
        # assert
        mocked_get.assert_called_with('http://company.com/Doe/May')
        self.assertEqual(schedule, 'Success')
        
        # Test a bad response
        mocked_get.return_value.ok = False
        
        # call function
        schedule = self.emp_1.monthly_schedule('June')
        
        # assert
        mocked_get.assert_called_with('http://company.com/Doe/June')
        
        # return_value.text is not necessary as func automatically returns 'Bad Response!' if .ok returns False
        self.assertEqual(schedule, 'Bad Response!')
```

****

**Tips & Best Practices**

- Consider a test-driven development approach where tests are designed before code is written
- Tests should be isolated and should be independent from other tests/dependencies
    - Consider mocking data read/write, API calls and external functions/dependencies
- Tests don't necessarily run in order
- Pick a single, specific functionality to verify, i.e. *start really small*
- Use available tools to get everything else out of the way
- In an existing codebase, don't try to write all the tests at once (*too overwhelming*)
- Write tests as early as they can be valuable

****

**References**

- [`unittest debug()` - `assert` docs](https://docs.python.org/3/library/unittest.html#unittest.TestCase.debug)
- [Unit Testing for Data Scientists](https://youtu.be/Da-FL_1i6ps)

****

## Basics of `pytest`

Consider a function that adds a column to a `DataFrame`:

```python
def add_col(df, new_col_name, default_value):
    df[new_col_name] = default_value
    return df
```

Here's a unit-test for this function:

```python
def test_add_col_passes():
    # setup
    df = pd.DataFrame({
        'col_a': ['a', 'a', 'a'],
        'col_b': ['b', 'b', 'b'],
        'col_c': ['c', 'c', 'c'],
    })
    
    # call function
    actual = add_col(df, 'col_d', 'd')
    
    # set expectations
    expected = pd.DataFrame({
        'col_a': ['a', 'a', 'a'],
        'col_b': ['b', 'b', 'b'],
        'col_c': ['c', 'c', 'c'],
        'col_d': ['d', 'd', 'd']
    })
    
    # assertion
    pd.testing.assert_frame_equal(actual, expected)
```

****

Using fixtures can help to modularize setup and teardown of a test:

```python
# Consider defining fixtures in a seperate fixture file and import it into `conftest.py` file
@pytest.fixture()
def df():
    return pd.DataFrame({
        'col_a': ['a', 'a', 'a'],
        'col_b': ['b', 'b', 'b'],
        'col_c': ['c', 'c', 'c'],
    })

@pytest.fixture()
def df_with_col_d():
    return pd.DataFrame({
        'col_a': ['a', 'a', 'a'],
        'col_b': ['b', 'b', 'b'],
        'col_c': ['c', 'c', 'c'],
        'col_d': ['d', 'd', 'd']
    })
````

The unit-test with the fixture will now be simplified:

```python
def test_add_col_passes_with_fixtures(df, df_with_col_d):
    actual = add_col(df, 'col_d', 'd')
    expected = df_with_col_d
    
    pd.testing.assert_from_equal(actual, expected)
```

****

We can run unit-tests without actually interacting with production databases by using **mocking** instead.

Here, we have a function that generates features by reading from a database

```python
# Read from a database
def generate_features(creds):
    # return some fake data from some database
    eng = sqlalchemy.create_engine('fake_connection_string')
    df = pd.read_sql('select col1, col2 from data_table', con=eng)
    
    # some processing to get features
    # ...
    
    return features
```

Here, we patch these functions with mock objects instead of actually interacting with the database.

```python
# Patch functions that we don't actually want to run and use mock objects instead
@mock.patch('pytest_examples.functions_to_test.sqlalchemy.create_engine')
@mock.patch('pytest_examples.functions_to_test.pd.read_sql')
def test_generate_features(read_sql_mock, engine_mock, creds, df):
    # set a return value here as DataFrame
    read_sql_mock.return_value = df
    
    # call function 
    actual_features = generate_features(creds)
    
    # assertion
    pd.testing.assert_frame_equal(actual_features, df)
```

****

# Basic code
A `minimal, reproducible example`